# Create a VGG network

In this notebook, we will focus on building a [VGG network](https://towardsdatascience.com/vgg-neural-networks-the-next-step-after-alexnet-3f91fa9ffe2c) (Visual Geometry Group) network, a highly influential architecture that follows the principles laid out in the progression from AlexNet by using deeper and smaller convolutional layers to enhance image classification performance. The VGG network is characterized by its simplicity, using mostly 3x3 convolutional layers stacked on top of each other in increasing depth

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import utils

## Create named-variables dynamically

Let's explore the practical application of the Python `vars()` function. This utility function is particularly useful when we need to manage or manipulate multiple similarly named variables dynamically within a class. It provides a dictionary representation of an object’s attributes, making it possible to access and modify them programmatically, which is an efficient way to handle attribute assignments in bulk.

In [ ]:
# Define a small class MyClass
class MyClass:
    def __init__(self):
        # One class variable 'a' is set to 1
        self.var1 = 1

# Create an object of type MyClass()
my_obj = MyClass()

In Python, classes possess an attribute known as `__dict__`. This attribute is a dictionary that holds the object's instance variables along with their corresponding values, structured as key-value pairs. This makes it possible to access, modify, or add new properties dynamically to instances of the class.

In [ ]:
my_obj.__dict__

{'var1': 1}

When we use the `vars()` function and pass in an object, it retrieves the object's `__dict__ attribute`. This attribute is essentially a Python dictionary that stores all of the object's instance variables along with their corresponding values, neatly organized as key-value pairs. This allows for straightforward access and manipulation of the object's properties.

In [ ]:
vars(my_obj)

{'var1': 1}

Typically, when we need to add a new variable to an object in Python, we might directly set it on the object. This straightforward method involves simply assigning a value to a new attribute on an instance of a `class`.

In [ ]:
# Add a new instance variable and give it a value
my_obj.var2 = 2

# Calls vars() again to see the object's instance variables
vars(my_obj)

{'var1': 1, 'var2': 2}

Another method to add an instance variable to an object involves using the `vars()` function. This function retrieves the `__dict__` attribute of the object, which is a dictionary containing all instance variables. We can then modify this dictionary directly using square bracket notation. For example, if we want to add a new variable named `var3` with a value of 3, we would use the syntax `vars(my_obj)['var3'] = 3`. This effectively adds a new key-value pair to the object's attribute dictionary, dynamically adding or modifying its properties.

In [ ]:
# Call vars, passing in the object.  Then access the __dict__ dictionary using square brackets
vars(my_obj)['var3'] = 3

# Call vars() to see the object's instance variables
vars(my_obj)

{'var1': 1, 'var2': 2, 'var3': 3}

#### Why this is helpful!
Understanding the utility of using `vars()` to manage object instance variables can be particularly beneficial in scenarios where we need to dynamically handle multiple similarly named variables. Here's why this approach is helpful:

1. **Dynamic Variable Names:** By using `vars()`, we gain the ability to pass in the name of the variable as a string. This flexibility is crucial when we need to dynamically access or modify attributes based on their names, which might not be predetermined in our code.
2. **Handling Sequential Variables:** Consider a situation where our object needs to handle a sequence of similarly named variables like var4, var5, ..., var9. Using `vars()` allows we to easily generate or access these variables in a loop, where the variable's name can be constructed as a string dynamically. This is much more convenient than manually coding each variable individually, especially as the number of variables grows.
3. **Scalable and Flexible Code:** This method makes our code more scalable and adaptable to changes. For example, if we decide to add more variables or change the naming convention, we can easily adjust the loop or the string manipulation logic without needing to overhaul the codebase.

This approach can significantly simplify the process of managing multiple variables, making our code cleaner and more maintainable, particularly in applications requiring extensive data manipulation or configuration.

In [ ]:
# Use a for loop to increment the index 'i'
for i in range(4,10):
    # Format a string that is var
    vars(my_obj)[f'var{i}'] = 0

# View the object's instance variables!
vars(my_obj)

{'var1': 1,
 'var2': 2,
 'var3': 3,
 'var4': 0,
 'var5': 0,
 'var6': 0,
 'var7': 0,
 'var8': 0,
 'var9': 0}

In Python, formatting strings can be accomplished in several efficient ways, each providing its own advantages depending on the context and Python version. Here are two common methods:

- **F-strings:** Introduced in Python 3.6, f-strings offer a very readable, concise, and convenient way to embed expressions inside string literals, using curly braces {}. The syntax involves prefixing the string with an f or F. For example, if we want to dynamically generate string names in a loop, we can use f"var{i}", where i is the loop variable. This method is both fast and easy to read, making it a popular choice for Python developers.
- **format method:** This method has been available since Python 2.6 and is useful across a wider range of Python versions, including those not supporting f-strings. The format method involves writing placeholder braces {} within the string and then calling `.format()` with the values or variables that should replace these placeholders. For example, `"var{}".format(i)` achieves the same outcome as the f-string method but is slightly more verbose.

Both methods are highly effective for creating formatted strings and can be chosen based on the specific requirements of the codebase or personal preference for syntax style.

In [ ]:
# Format a string using f-string notation
i=1
print(f"var{i}")

# Format a string using .format notation
i=2
print("var{}".format(i))

var1
var2


Variables of class can also be acccessed inside the class definition using `vars(self)`

In [ ]:
# Define a small class MyClass
class MyClass:
    def __init__(self):
        # Use vars(self) to access the class's dictionary of variables
        vars(self)['var1'] = 1

# Create an object of type MyClass()
my_obj = MyClass()
vars(my_obj)

{'var1': 1}

## Create a generic VGG block

Now we're going to dive into how we can modularize the construction of a VGG network by defining a Block class, capable of generating customizable blocks of Conv2D layers followed by a MaxPool2D layer. This approach allows us to encapsulate repeated patterns within the network, facilitating both scalability and ease of modifications.

The Block class is designed to create a sequence of convolutional layers, each followed by a ReLU activation, and conclude the sequence with a max pooling operation to reduce spatial dimensions and prepare the output for the next block or layer.

#### Constructor (`__init__`)
- **Parameters Storage:** The constructor stores essential parameters like the number of filters, kernel size, and the number of repetitions for the convolutional layers. These parameters are kept as class variables to be used later in the call method.
-**Conv2D Layers Definition:** Utilizing a loop, the constructor dynamically creates several Conv2D layers based on the specified number of repetitions. Each layer is named systematically using string formatting (e.g., conv2D_0, conv2D_1), facilitating organized creation and later access.
- ** MaxPool2D Layer:** Following the convolutional sequence, a MaxPool2D layer is defined with specific pool size and stride, which helps in reducing the feature map dimensions and adding translational invariance.


#### Call Method (call)
- **Layer Connection:** The call method sequentially connects these layers. It starts by connecting the input to the first Conv2D layer and then iteratively connects each subsequent Conv2D layer to the previous one.
- **Pooling Integration:** After all convolutional layers are connected, the output is then passed through the MaxPool2D layer.
- **Output:** The final output after the MaxPool2D layer is returned, ready to be passed to the next block or layer in the larger model.
Why This Approach is Beneficial


This class-based approach allows achieiving modularity as it encapsulates a repetitive process (creating multiple Conv2D layers) into a single block, making the main model definition cleaner and more manageable. Blocks can be instantiated with different parameters to build varied structures within the network, making this approach extremely flexible and powerful for designing complex architectures. Furthermore, by isolating the repetitive parts of the network construction into a block, the overall code becomes more readable and easier to maintain, especially when adjustments are needed.

This method of constructing neural network architectures showcases how advanced features of object-oriented programming can be leveraged to build sophisticated and customizable neural network architectures efficiently.

In [ ]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions

        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions+1):

            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters, self.kernel_size, padding='same', activation='relu')

        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPool2D(pool_size, strides=strides)

    def call(self, inputs):
        # Access the class's conv2D_0 layer
        conv2D_0 = vars(self)['conv2D_0']

        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # For the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1, self.repetitions+1):
            # Access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']

            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)

        return max_pool

## Create the Custom VGG network

We're going to assemble a model composed of several VGG-like blocks, constructed using the `Block` class we previously established. Each block will contribute to a progressively deeper and more complex feature extraction network, culminating in a classification layer.

##### Constructor (`__init__`)
- **Block Parameters:** The constructor of each Block instance requires specific parameters like the number of filters, kernel size, and the number of repetitions. While kernel size and strides might remain default for simplicity, the specifications for each block will vary to capture increasingly complex features as we move deeper into the network.
- **Defining Blocks:**
  * block_a: Configured with 64 filters, a kernel size of 3, and 2 repetitions.
  * block_b: Set with 128 filters, a kernel size of 3, and 2 repetitions.
  * block_c: Composed of 256 filters, a kernel size of 3, and 3 repetitions.
  * block_d: Contains 512 filters, a kernel size of 3, and 3 repetitions.
  * block_e: Also has 512 filters, a kernel size of 3, and 3 repetitions.


Following block 'e', additional layers will form the decision-making head of the model:

- **Flatten Layer:** Transforms the 3D feature maps to 1D feature vectors.
- **Fully Connected Layer (fc):** A dense layer with 256 units and 'relu' activation to introduce non-linearity and aid in learning complex patterns.
- **Classifier Layer:** The final dense layer, using 'softmax' activation, translates the learned features into probabilities across the classes.


##### Call Method (call)
- **Layer Connection:** Here, we sequentially connect the input through each defined block and layer:
  * Start with the inputs.
  * Feed into block_a, then to block_b, block_c, block_d, and finally block_e.
  * Pass the output from block_e through the flatten layer to prepare for dense connections.
  * Continue through the fc layer to shape the deep features.
  * Culminate with the classifier layer where the final classification is computed.
  * The output of the call method is the output from the classifier layer, which provides the class probabilities for the input data.

In [ ]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the (filters, kernel_size, repetitions) configurations
        self.block_a = Block(64, 3, 2)
        self.block_b = Block(128, 3, 2)
        self.block_c = Block(256, 3, 3)
        self.block_d = Block(512, 3, 3)
        self.block_e = Block(512, 3, 3)

        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = tf.keras.layers.Dense(units=256, activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(units=num_classes, activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

### Load data and train the VGG network

In our next steps, we will explore how to load the dataset and prepare our VGG network for training. We will use `cats_vs_dogs` dataset that is provided to us in TensorFlow Datasets `tfds`

In [ ]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes
vgg = MyVGG(num_classes=2)

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=10)

Epoch 1/10
727/727 [==============================] - 145s 190ms/step - loss: 0.6905 - accuracy: 0.5331
Epoch 2/10
727/727 [==============================] - 134s 184ms/step - loss: 0.6765 - accuracy: 0.5840
Epoch 3/10
727/727 [==============================] - 143s 196ms/step - loss: 0.6637 - accuracy: 0.6082
Epoch 4/10
727/727 [==============================] - 139s 192ms/step - loss: 0.6504 - accuracy: 0.6284
Epoch 5/10
727/727 [==============================] - 141s 194ms/step - loss: 0.6365 - accuracy: 0.6488
Epoch 6/10
727/727 [==============================] - 141s 193ms/step - loss: 0.6234 - accuracy: 0.6633
Epoch 7/10
727/727 [==============================] - 142s 195ms/step - loss: 0.6119 - accuracy: 0.6740
Epoch 8/10
727/727 [==============================] - 143s 197ms/step - loss: 0.6021 - accuracy: 0.6833
Epoch 9/10
727/727 [==============================] - 140s 192ms/step - loss: 0.5938 - accuracy: 0.6870
Epoch 10/10
727/727 [==============================] - 144s 198m